In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'top_model_weights_path.h5'

In [3]:
epochs = 10
batch_size = 24
split=0.9

In [4]:
X_train=np.load('X_train_rotate.npy')
target_train=np.load('target_train.npy')

In [5]:
def get_callbacks(filepath, patience):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [7]:
def model_plus(lr,beta1):
    base_model = applications.VGG16(weights='imagenet', include_top=False,input_shape=(75,75,3))
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(1024, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))
    # note that it is necessary to start with a fully-trained
    # classifier, including the top classifier,
    # in order to successfully do fine-tuning
    top_model.load_weights(top_model_weights_path)
    # add the model on top of the convolutional base
    #model.add(top_model)
    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers[:-5]:
        layer.trainable = False
    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    mypotim=Adam(lr=lr, beta_1=beta1, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    return model

In [8]:
file_path = "model_weights.hdf5"

In [10]:
#5fold cross validation

from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from keras import initializers
weight_init = initializers.glorot_uniform(seed=7)

seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=seed)

for lr in [1e-4,1e-5,1e-6]:
    for beta1 in [0.5,0.7,0.9]:
        cvscores = []

        for train, test in kfold.split(X_train, target_train):
            model = model_plus(lr,beta1)
            model.fit(X_train[train], target_train[train],
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=0,
                    validation_data=(X_train[test], target_train[test]),
                    callbacks = get_callbacks(file_path,5))
            model.load_weights(filepath=file_path)
            scores = model.evaluate(X_train[test], target_train[test], verbose=0)
            print(model.metrics_names[0], scores[0])
            print(model.metrics_names[1], scores[1])
            cvscores.append(scores[0])
        print('lr=',lr,'beta1',beta1,'Test loss average================',np.mean(cvscores), 'Test loss std:',np.std(cvscores))

loss 0.2698023683
acc 0.893042575286
loss 0.341593784861
acc 0.885773624153
loss 0.316785032937
acc 0.860851505711
loss 0.282813037164
acc 0.881496881497
loss 0.258948403919
acc 0.879292403746
lr= 0.0001 beta1 0.5 Test loss average================ 0.293988525437 Test loss std: 0.0307363411297
loss 0.280352931826
acc 0.889927310488
loss 0.263163816582
acc 0.886812045691
loss 0.34904452329
acc 0.835929387331
loss 0.281125417082
acc 0.878378378378
loss 0.277134961147
acc 0.875130072841
lr= 0.0001 beta1 0.7 Test loss average================ 0.290164329986 Test loss std: 0.0301450985079
loss 0.247362203858
acc 0.899273104881
loss 0.257320971813
acc 0.882658359356
loss 0.323489382004
acc 0.860851505711
loss 0.235581006796
acc 0.891891891892
loss 0.282847129652
acc 0.888657648283
lr= 0.0001 beta1 0.9 Test loss average================ 0.269320138825 Test loss std: 0.031246588084
loss 0.208867745383
acc 0.909657320872
loss 0.21845103367
acc 0.894080996885
loss 0.230069976776
acc 0.907580477674


In [ ]:
print('Test loss average:',np.mean(cvscores), 'Test loss std:',np.std(cvscores))
print('Test acc average:',np.mean(cvscores_), 'Test loss std:',np.std(cvscores_))

In [ ]:
'print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
X_test=np.load('X_test.npy')
predicted_test=model.predict(X_test,verbose=1)

In [ ]:
import pandas as pd
submission = pd.DataFrame()
test_id=np.load('test_id.npy')
submission['id']=test_id
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

In [ ]:
i=0
for layer in model.layers:
    i=i+1
    
print (i)    